In [92]:
import requests
import pandas as pd
from config import weather_api_key

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [93]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Metric&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Metric&APPID=914d1f676bc005f30fced2fbdc7c3f54


In [94]:
city_url = url + "&q=" + "Toronto"
print(city_url)

http://api.openweathermap.org/data/2.5/weather?units=Metric&APPID=914d1f676bc005f30fced2fbdc7c3f54&q=Toronto


In [95]:
weather_response = requests.get(city_url)
boston_data = weather_response.json()
if weather_response.status_code == 200:
    print(f"City Weather found.")
else:
    print(f"City weather not found.")


City Weather found.


In [96]:
lat = boston_data["coord"]["lat"]
lng = boston_data["coord"]["lon"]
max_temp = boston_data["main"]["temp_max"]
humidity = boston_data["main"]["humidity"]
clouds = boston_data["clouds"]["all"]
wind = boston_data["wind"]["speed"]
print(lat, lng, max_temp, humidity, clouds, wind)

43.7001 -79.4163 29.92 49 1 0.89


In [97]:
# Import the datetime module from the datetime library.
from datetime import datetime
# Get the date from the JSON file.
date = boston_data["dt"]
# Convert the UTC date to a date format with year, month, day, hours, minutes, and seconds.
datetime.utcfromtimestamp(date)

datetime.datetime(2021, 8, 5, 19, 33, 44)

======================================================

In [98]:
# Import the datetime module from the datetime library.
from datetime import datetime

In [99]:
# Create an empty list to hold the weather data.
city_data = []

cities = ["Toronto", "Pittsburgh", "Austin", "New York", "Los Angeles", "Seattle"]
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [100]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | Toronto
Processing Record 2 of Set 1 | Pittsburgh
Processing Record 3 of Set 1 | Austin
Processing Record 4 of Set 1 | New York
Processing Record 5 of Set 1 | Los Angeles
Processing Record 6 of Set 1 | Seattle
-----------------------------
Data Retrieval Complete      
-----------------------------


In [101]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Toronto,43.7001,-79.4163,29.92,49,1,0.89,CA,2021-08-05 19:33:44
1,Pittsburgh,40.4406,-79.9959,30.67,48,1,1.34,US,2021-08-05 19:38:52
2,Austin,30.2672,-97.7431,29.94,87,75,2.57,US,2021-08-05 19:38:45
3,New York,40.7143,-74.0060,29.16,62,90,1.54,US,2021-08-05 19:37:09
4,Los Angeles,34.0522,-118.2437,43.63,51,1,0.00,US,2021-08-05 19:38:46
5,Seattle,47.6062,-122.3321,29.82,51,75,0.00,US,2021-08-05 19:37:01


In [102]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")